# Direct Access ABoVE UAVSAR dataset

In this tutorial, we will read the Active Layer Thickness from Airborne L- and P- band SAR dataset (https://doi.org/10.3334/ORNLDAAC/2004) from NASA Earthdata Cloud.

In [1]:
import earthaccess
import s3fs
import xarray as xr

## Authentication
We recommend authenticating your Earthdata Login (EDL) information using the `earthaccess` python library as follows:

In [2]:
# works if the EDL login already been persisted to a netrc
auth = earthaccess.login(strategy="netrc")
if not auth.authenticated:
    # ask for EDL credentials and persist them in a .netrc file
    auth = earthaccess.login(strategy="interactive", persist=True)

## Search granules

In [3]:
doi = '10.3334/ORNLDAAC/2004' # uavsar data
granules = earthaccess.search_data(
    count=-1, # needed to retrieve all granules
    granule_name = f"*.nc4", # retrieve only netcdfs
    cloud_hosted=True, # make sure dataset is in cloud for direct access
    doi=doi
)
print(f'Granules found: {len(granules)}')

Granules found: 51


## S3 Authentication

In [4]:
s3credentials = earthaccess.get_s3_credentials("ORNLDAAC")
fs = s3fs.S3FileSystem(anon=False,
                       key=s3credentials['accessKeyId'],
                       secret=s3credentials['secretAccessKey'],
                       token=s3credentials['sessionToken'])

## Reading the first granule

In [5]:
s3_url = granules[0].data_links(access='direct')[0]
ds = xr.open_dataset(fs.open(s3_url, 'rb'), engine='h5netcdf', chunks={})
ds

<xarray.Dataset>
Dimensions:  (x: 2827, y: 2959)
Coordinates:
  * x        (x) float64 -1.358e+06 -1.358e+06 ... -1.273e+06 -1.273e+06
  * y        (y) float64 2.556e+06 2.556e+06 2.556e+06 ... 2.467e+06 2.467e+06
    lat      (y, x) float64 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    lon      (y, x) float64 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
Data variables: (12/14)
    crs      |S1 ...
    alt      (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    alt_unc  (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    sub      (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    sub_unc  (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    Sw0      (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    ...       ...
    wtd_unc  (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    mv_6cm   (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    mv_12cm  (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    mv_20cm  (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    mv_alt   (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
    qa       (y, x) float32 dask.array<chunksize=(2959, 2827), meta=np.ndarray>
Attributes:
    title:            Active Layer Thickness Derived from Airborne L- and P-b...
    source:           The Airborne InSAR and PolSAR Permafrost Dynamics Obser...
    project:          Arctic-Boreal Vulnerability Experiment (ABoVE)
    Conventions:      CF-1.6
    summary:          This data file provides estimates of seasonal subsidenc...
    references:       Chen et al. 2018 https://doi.org/10.1109/IGARSS.2018.85...
    publisher_name:   Oak Ridge National Laboratory (ORNL) Distributed Active...
    publisher_url:    https://daac.ornl.gov
    publisher_email:  uso@daac.ornl.gov
    citation:         Chen, R.H., R.J. Michaelides, J. Chen, A.C. Chen, L.K. ...